In [1]:
import nltk
import re, string
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import twitter_samples
from nltk import FreqDist
import random
from nltk import classify
from nltk import NaiveBayesClassifier

positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
stop_words = stopwords.words('english')

In [2]:
from pandas import DataFrame

In [7]:
df1 = DataFrame (positive_tweets,columns=['tweets'])

In [13]:
df2 =DataFrame (negative_tweets,columns=['tweets'])

In [14]:
df1.shape

(5000, 1)

In [15]:
df1.to_csv('df1.csv')

In [16]:
df2.to_csv('df2.csv')

In [4]:
import pandas as pd

In [284]:
df_p=pd.read_csv('df1.csv')

In [285]:
df_n=pd.read_csv('df2.csv')

In [351]:
df=pd.concat([df_p,df_n])
df.to_csv('df.csv')
#we add the sentiment column after download it

In [5]:
df=pd.read_csv('df.csv')

In [6]:
df=df.dropna()

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 10001
Data columns (total 4 columns):
Unnamed: 0      10000 non-null int64
Unnamed: 0.1    10000 non-null object
tweets          10000 non-null object
sentiment       10000 non-null object
dtypes: int64(1), object(3)
memory usage: 390.6+ KB


In [8]:
def remove_URL(headline_text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', headline_text)

In [9]:
df['tweets']=df['tweets'].apply(remove_URL)

In [10]:
def remove_html(headline_text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',headline_text)

df['tweets']=df['tweets'].apply(remove_html)


In [11]:
import re, string

In [12]:
def remove_punct(headline_text):
    table=str.maketrans('','',string.punctuation)
    return headline_text.translate(table)
df['tweets']=df['tweets'].apply(remove_punct)

In [13]:
def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return df['tweets']==df1['tweets'].apply(remove_stopwords)

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import nltk
normalization = None
normalization = 'stemmer'
normalization = 'lemmatizer'
def stem_tokens(tokens):
    stemmer = nltk.stem.PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    return tokens
def lemmatize_tokens(tokens):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens
def normalize_tokens(normalization):
    if normalization is not None:
        if normalization == 'stemmer':
            df['tweets']==df['tweets'].apply(stem_tokens)
        elif normalization == 'lemmatizer':
            df['tweets']==df['tweets'].apply(lemmatize_tokens)
        
normalize_tokens(normalization)

In [15]:
df = df[["tweets", "sentiment"]]
df.head()

,tweets,sentiment
0,FollowFriday FranceInte PKuchly57 MilipolParis...,Positive
1,Lamb2ja Hey James How odd Please call our Con...,Positive
2,DespiteOfficial we had a listen last night As...,Positive
3,97sides CONGRATS,Positive
4,yeaaaah yippppy my accnt verified rqst has su...,Positive


In [16]:
def getFeatureVector(dataset):
    featureVector = []

    words = dataset.split() # split tweet into words
    for w in words:
        
        w = replaceTwoOrMore(w)  # replace two or more with two occurrences
        
        w = w.strip('\'"?,.') # strip punctuation
        
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w) # check if stats with an alphabet
        
        if(w in stopWords or val is None): # ignore stop word
            continue
        else:
            featureVector.append(w.lower())
    return featureVector

In [17]:
def featureExtraction():
    featureVector = getFeatureVector(dataset)
    dataset.append((featureVector, sentiment))
    return dataset

In [18]:
def get_words_in_tweets(dataset):
    all_words = []
    for (text, sentiment) in dataset:
        all_words.extend(text)
    return all_words

In [21]:
def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist) # frequency distrubtion of all words
    word_features = wordlist.keys()
    return word_features

#word_features = get_word_features(get_words_in_tweets(dataset))

In [22]:
def extract_features(dataset):
    settweet = set(dataset)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in settweet)
    return features

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

In [24]:
X = df["tweets"]
y = df["sentiment"]
#count_vect = CountVectorizer()
#X=vectorizer.fit_transform(X).toarray()
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=42)

count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(X_train)

In [25]:
from sklearn.feature_extraction.text import TfidfTransformer

# create an instance:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.fit_transform(X_test)

In [30]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [33]:
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train_tfidf, y_train)
from sklearn.pipeline import Pipeline
                    # vectorize                    # classifier
text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])
text_clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
               

In [34]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [35]:
predictions = text_clf.predict(X_test)

from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test, predictions))

[[756 232]
 [234 778]]


In [36]:
print(classification_report(y_test, predictions))
# does vectorizing the tweets get a better performing classifier?
# -- COMPARE THESE RESULTS TO THE NAIVE BAYES precision, recall etc ---

              precision    recall  f1-score   support

    Negative       0.76      0.77      0.76       988
    Positive       0.77      0.77      0.77      1012

    accuracy                           0.77      2000
   macro avg       0.77      0.77      0.77      2000
weighted avg       0.77      0.77      0.77      2000



In [37]:
from sklearn import metrics
metrics.accuracy_score(y_test, predictions)


0.767

In [38]:
# pip_install tweepy
# pip_install pandas
# pip_install numpy
# pip install matplotlib
# pip install textblob

from textblob import TextBlob
import tweepy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# Our Twitter API Credentials:
# (Normally, we store these in a seperate file for precautionary measures.
#  However, for the sake of simplicity, we are providing them here.)
Consumer_Key = 'j1PZszWJPTn50m9CyZaKQEL4V'
Consumer_Secret = '0VLZXNHBidBXOYELBSmAJnWpsohTlMQ2flop7cGY4IRd7gRPc8'        
Access_Token = '1334510916344360963-6ESgNupKmKn8yuCEEpi91XLO7cXttC'    
Access_Token_Secret = 'O4ghcRniTGXb9V7LIcfW5CpK93Pc6yfTiRY5vxSZReHuq'

class Twitter_Customer():

    def __init__(self, twitter_user = None):
        self.auth = Authentication().authenticate_twitter_app()
        self.twitter_client = tweepy.API(self.auth)
        self.twitter_user = twitter_user

    def twitter_customer_api(self):
        return self.twitter_client

    
class Authentication():

    def authenticate_twitter_app(self):
        auth = tweepy.OAuthHandler(Consumer_Key, Consumer_Secret)
        auth.set_access_token(Access_Token, Access_Token_Secret)
        return auth

    
class My_Stream_Listener():

    def __init__(self):
        self.twitter_autenticator = Authentication()    

    def stream_tweets(self, fetched_tweets_filename):
        listener = TwitterListener(fetched_tweets_filename)
        auth = self.twitter_autenticator.authenticate_twitter_app() 
        stream = tweepy.Stream(auth, listener)
        
class TwitterListener(tweepy.StreamListener):

    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          
    def on_error(self, status):
        if status == 420:
            return False
        print(status)
        
class TweetAnalyzer():

    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|(RT[\s]+)", " ", tweet).split())

    def analyze_sentiment_polarity(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        
        if analysis.sentiment.polarity < 0:
            return 'Negative'
        elif analysis.sentiment.polarity == 0:
            return 'Neutral'
        else:
            return 'Positive'
    
    def analyze_sentiment_subjectivity(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        return round(analysis.sentiment.subjectivity, 2)

    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['tweets'])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['len'] = np.array([len(tweet.text) for tweet in tweets])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df['retweets'] = np.array([tweet.retweet_count for tweet in tweets])
        return df
 
if __name__ == '__main__':

    twitter_client = Twitter_Customer()
    tweet_analyzer = TweetAnalyzer()

    api = twitter_client.twitter_customer_api()

    #tweets = api.user_timeline(screen_name = "TheRock", count=200)
    tweets = api.search(q = "#Apple",count = 10000, lang = "en")
    
    df_tweet_api = tweet_analyzer.tweets_to_data_frame(tweets)
    df_tweet_api['sentiment'] = np.array([tweet_analyzer.analyze_sentiment_polarity(tweet) for tweet in df_tweet_api['tweets']])
    df_tweet_api['subjectivity'] = np.array([tweet_analyzer.analyze_sentiment_subjectivity(tweet) for tweet in df_tweet_api['tweets']])
    df_tweet_api.to_csv('twitter_data.csv')

df_tweet_api = pd.read_csv('twitter_data.csv')
df_tweet_api.head()

,Unnamed: 0,tweets,date,len,likes,retweets,sentiment,subjectivity
0,0,RT @aaple_lab: #iPhone 13 (12s) Pro \n- Design...,2020-12-14 03:22:16,139,0,9,Neutral,0.12
1,1,"RT @TrueWildernesss: #Apple wand, #Swirl apple...",2020-12-14 03:22:01,139,0,1,Negative,0.50
2,2,"#Apple wand, #Swirl apple wand, Energized wood...",2020-12-14 03:21:47,135,0,1,Negative,0.50
3,3,RT @aaple_lab: #iPhone 13 (12s) Pro \n- Design...,2020-12-14 03:20:46,139,0,9,Neutral,0.12
4,4,RT @LWNovels: Every chapter is a thriller. TH...,2020-12-14 03:20:25,140,0,1,Positive,0.00


In [39]:
df_tweet_api=df_tweet_api[df_tweet_api['sentiment']!='Neutral']

In [40]:
df_tweet_api.shape

(55, 8)

In [41]:
def remove_URL(headline_text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', headline_text)

In [42]:
df_tweet_api['tweets']=df_tweet_api['tweets'].apply(remove_URL)

In [43]:
df_tweet_api['tweets']=df_tweet_api['tweets'].apply(remove_html)

In [44]:
df_tweet_api['tweets']=df_tweet_api['tweets'].apply(remove_punct)

In [45]:
def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return df_tweet_api['tweets']==df1['tweets'].apply(remove_stopwords)

In [46]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import nltk
normalization = None
normalization = 'stemmer'
normalization = 'lemmatizer'
def stem_tokens(tokens):
    stemmer = nltk.stem.PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    return tokens
def lemmatize_tokens(tokens):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens
def normalize_tokens(normalization):
    if normalization is not None:
        if normalization == 'stemmer':
            df_tweet_api['tweets']==df_tweet_api['tweets'].apply(stem_tokens)
        elif normalization == 'lemmatizer':
            df_tweet_api['tweets']==df_tweet_api['tweets'].apply(lemmatize_tokens)
        
normalize_tokens(normalization)

In [47]:
x2_test=df_tweet_api['tweets']
y2_test=df_tweet_api['sentiment']

In [48]:
prediciton = text_clf.predict(x2_test)

from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(prediciton,y2_test))

[[ 8  4]
 [12 31]]


In [49]:
print(classification_report(y2_test, prediciton))

              precision    recall  f1-score   support

    Negative       0.67      0.40      0.50        20
    Positive       0.72      0.89      0.79        35

    accuracy                           0.71        55
   macro avg       0.69      0.64      0.65        55
weighted avg       0.70      0.71      0.69        55



In [50]:
metrics.accuracy_score(y2_test, prediciton)

0.7090909090909091